In [ ]:
import numpy as np

from module_.dataLoader import dataLoader

episodes, transitions, labels = dataLoader('testbed')

sensors = set()
for e in episodes:
    sensors = sensors | set(e[:,0])
sensors = sorted(sensors)

In [ ]:
i = 3
episode, transition, label = episodes[i], transitions[i], labels[i]

start_time, end_time = int(episode[0,2]), int(episode[-1,2])

active_log = np.zeros((end_time-start_time+1, len(sensors)))

on_interval = {sensor: [] for sensor in sensors}

active_states = {sensor: False for sensor in sensors}
active_startt = {sensor: 0 for sensor in sensors}

# test = []

for i in range(len(episode)):
    s, v, t = episode[i]
    t = int(t)-start_time
    if s[0]!="M": 
        # test.append(episode[i])
        continue

    if v=="true":
        if active_states[s]==False: # False -> True (State 변경)
            active_startt[s]=t
            active_states[s]=True

            # test.append(episode[i])
        else:                       # True -> True (이전 True까지 Update)
            active_log[active_startt[s]:t,sensors.index(s)]=1
            on_interval[s].append((active_startt[s], t))
            active_startt[s]=t
    else:
        if active_states[s]==True:  # True -> False (이전 True까지 Update하고 초기화)
            active_log[active_startt[s]:t,sensors.index(s)]=1
            on_interval[s].append((active_startt[s], t))
            active_startt[s]=t
            active_states[s]=False

            # test.append(episode[i])
    
    for sensor in sensors:
        if active_states[sensor] and t-active_startt[sensor]>40: # True -> ... (True 이후 이벤트가 없는 경우 - Update하고 초기화)
            active_log[active_startt[sensor]:active_startt[sensor]+40,sensors.index(sensor)]=1
            on_interval[sensor].append((active_startt[sensor], active_startt[sensor]+40))
            active_states[sensor]=False

            # test.append(
            #     np.array([sensor, "false", t+start_time])
            # )

    if i==len(episode)-1:
        for sensor in sensors:
            if active_states[sensor]:
                active_log[active_startt[sensor]:,sensors.index(sensor)]=1
                on_interval[sensor].append((active_startt[sensor], t))
                active_states[sensor]=False

                # test.append(
                #     np.array([sensor, "false", t+start_time])
                # )

print(label, float(episode[transition-1,2])-start_time, float(episode[transition,2])-start_time)

In [ ]:
on_vector = {sensor: [] for sensor in sensors}

for k, v in on_interval.items():
    for lb, ub in v:
        on_vector[k].append(
            (lb, ub, k, np.mean(active_log[lb:ub, :], axis=0))
        )

In [ ]:
total = []
for v in on_vector.values():
    total.extend(v)
total = sorted(total, key=lambda x: x[1])

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

train_interval = 600
test_interval = 60

import collections

outlier_scores = []

train_lb, train_rb = 0, train_interval + test_interval
test_lb, test_rb = train_interval, train_interval + test_interval

# train_set = collections.defaultdict(list)

while test_rb<len(active_log):

    train_set = collections.defaultdict(list)
    test_set = collections.defaultdict(list)
    for item in total:
        if item[0]>=train_lb and item[1]<=train_rb:
            train_set[item[2]].append(item)
        if item[0]>=test_lb and item[1]<=test_rb:
            test_set[item[2]].append(item)

    # print(train_set)
    # print(test_set)
    
    avg_n_errors = []
    for sensor in train_set.keys():
        
        if len(train_set[sensor])>=3:

            vectors = np.array(
                [item[-1] for item in train_set[sensor]]
            )

            clf = LocalOutlierFactor(n_neighbors=3)
            y_pred = clf.fit_predict(vectors)
            # n_errors = y_pred[-len(test_set[sensor]):]
            # avg_n_errors.extend(list(n_errors))
            vector_scores = clf.negative_outlier_factor_
            

            normalized_scores = (vector_scores.max() - vector_scores) / (vector_scores.max() - vector_scores.min())
            test_scores = normalized_scores[-len(test_set[sensor]):]
            # avg_n_errors.extend(test_scores)
            avg_n_errors.append(test_scores.mean())
    
    oscores = sum(avg_n_errors)/len(avg_n_errors) if len(avg_n_errors) else 1.

    # print(test_lb, test_rb, oscores)
    outlier_scores.append((test_lb, oscores))

    train_lb += test_interval
    train_rb += test_interval

    test_lb += test_interval
    test_rb += test_interval
    

In [ ]:
from matplotlib import pyplot as plt

plt.plot(np.array(outlier_scores)[:,0]/60, np.array(outlier_scores)[:,1])

for i in range(len(np.array(outlier_scores))):
    if np.array(outlier_scores)[:,1][i]>0.8:
        print(i)

In [ ]:
active_log.shape # (duration, sensor_number)

on_interval = {sensor: [] for sensor in sensors}

for i in range(active_log.shape[1]):

    # 1. 센서 i의 ON 구간을 전부 찾는다
    # 2. 각 구간에서 다른 센서들의 ON 비율을 계산해서 [0, 1] 정규화된 벡터를 저장한다 (시작, 종료, 벡터)
    # 최종 결과물 - 각 센서별 ON 상태에서의 벡터 집합
    # 각 센서별로 클러스터링을 합니다.

    col = active_log[:, i]
    start_on = 0
    on_ing = False
    for j in range(len(col)):
        if col[j]==1:
            if not on_ing:
                start_on = j
                on_ing = True
        else:
            if on_ing:
                on_interval[sensors[i]].append((start_on, j-1))
                on_ing = False
        
        if j==len(col)-1:
            if on_ing:
                on_interval[sensors[i]].append((start_on, j))
                on_ing = False
